# ISODATA 算法实现

In [1]:
# 导入库
from sklearn import datasets
import numpy as np
import random
from sklearn.model_selection import train_test_split

In [2]:
#加载数据并显示
iris = datasets.load_iris()
[length, width] = iris.data.shape
print('iris dataset length:%d, width:%d' % (length,width))

iris dataset length:150, width:4


In [3]:
#定义Initial_Center类包含五种初始化方法
class Initial_Center():
    #初始化函数记录用于类别个数k
    #Random_Center对应随机选取数据中心的方法 整个数据集作为输入参数
    #Empircal_Center 对应随机选取数据中心的方法 以指定的数据中心作为输入参数
    #Random_MeanCenter 对应随机将数据分为K类取这K类数据的均值向量作为初始数据中心的方法 整个数据集作为输入参数
    #FarDistance_Center 选择批次距离尽可能远的点作为初始数据中心 整个数据集作为输入参数
    #Dense_Center 对应密度法求初始数据中心 
    #输入整个数据集， r表示计算密度的半径 Dt是用于选择高密度集的阈值
    #经过对数据的计算分析， iris数据集r取0.4，Dt取6较合适
    def __init__(self,k):
        self.n_class = k
        
    def Random_Center(self,dataset):
        [length, width] = dataset.shape
        indices = random.sample(range(0, length),self.n_class)
        self.initial_center  = dataset[indices]
      
    
    def Empirical_Center(self,Centers):
        self.initial_center = Centers
    
    def Random_MeanCenter(self,dataset):
        [length, width] = dataset.shape
        length = length - length % self.n_class
        
        indices = random.sample(range(0, length),length)
        self.initial_center  = np.mean(dataset[indices].reshape((self.n_class,-1,width)), 1)
        
    
    def FarDistance_Center(self,dataset):
        [length, width] = dataset.shape
        self.initial_center = np.zeros((self.n_class,width))
        self.indices = random.sample(range(0, length),self.n_class)
        self.indices[0] = np.random.randint(0,length)
        self.initial_center[0] = dataset[self.indices[0]]
        self.distance = np.sum(np.square(dataset - self.initial_center[0]),1)
            
        for i in range(self.n_class-1):
            self.distance += np.sum(np.square(dataset - self.initial_center[i]),1)
            indice = np.argmax(self.distance)
            #print(np.max(self.distance))
            #print(np.min(self.distance))
            if indice in self.indices:
                self.distance[indice] = 0
                
            self.indices[i+1] = np.argmax(self.distance)
            
        #print(sef.indices)   
        self.initial_center = dataset[self.indices]
       
    
    def Calcul_Dense(self,dataset,r):
        [length,width] = dataset.shape
        self.dense = np.zeros((length,))
        
        for i in range(length):
            decision = np.zeros((length,))
            temp_distance = np.sum(np.square(dataset - dataset[i]),1)
            decision[temp_distance <= r*r] = 1
            self.dense[i] = np.sum(decision)
            #print(self.dense[i]-1)
            
            
    def Dense_Center(self,dataset,r,Dt):
        [length, width] = dataset.shape
        self.initial_center = np.zeros((self.n_class,width))
        self.Calcul_Dense(dataset,r)
        dataset = dataset[self.dense > Dt]
        temp_matrix = self.dense[self.dense > Dt]
        #print(temp_matrix)
        self.indices = random.sample(range(0, length),self.n_class)
        self.indices[0] = np.argmax(temp_matrix)
        
        self.initial_center[0] = dataset[self.indices[0]]
        self.distance = np.sum(np.square(dataset - self.initial_center[0]),1)
            
        for i in range(self.n_class-1):
            self.distance += np.sum(np.square(dataset - self.initial_center[i]),1)
            indice = np.argmax(self.distance)
            #print(np.max(self.distance))
            #print(np.min(self.distance))
            if indice in self.indices:
                self.distance[indice] = 0
                
            self.indices[i+1] = np.argmax(self.distance)
            
        #print(sef.indices)   
        self.initial_center = dataset[self.indices]
    

In [4]:
class ISODATA():
    #__init__函数初始化ISODATA类的重要参数
        #centers -> Nc个初始聚类中心
        #k -> 预期的聚类中心数目
        #Theta_n -> 每一个聚类域中最少的样本数目，如果少于这个数目就不作为一个独立的聚类域
        #Theta_s -> 聚类域中样本距离分布的标准差
        #Theta_c -> 两聚类中心之间的最小距离，小于此距离，两个聚类合并
        #L -> 一次迭代运算中可以合并的聚类还在那个心的最多对数
        #r -> 分裂处理时，最大标准差分量取得比例
        #self.MeanDistanceInClass -> 记录每一类中各数据到其类中心的平均距离
        #self.MeansDistanceTotal -> 记录全部模式样本对其相应聚类中心的总平均距离
        
    #Labelling 函数按照最小距离准则给数据标签
    #Clustering 函数按照均值计算新的聚类中心
    #Operation_Division 分裂操作
    #Operation_Merging 合并操作
    #Interaction 函数作为人机交互接口，提供改变输入参数的功能

    def __init__(self,centers,Nc,k,Theta_n, Theta_s, Theta_c, L):
        self.centers = centers
        self.Nc = Nc
        self.n_class = k
        self.Theta_n = Theta_n
        self.Theta_s = Theta_s
        self.Theta_c = Theta_c
        self.L = L
        self.r = 0.5
        
        self.length = int(0)
        self.width = int(0)
        
        
    def Labelling(self,dataset):
        self.MeanDistanceInClass = np.zeros((self.Nc,))
        self.MeanDistanceTotal = 0.0
        self.num_per_class = np.zeros((self.Nc,))
        
        if self.centers.size == 0:
            print('parameters wrong!!! centers matrix is empty now!')
            
        [length,width] = dataset.shape
        self.length = length
        self.width = width
        self.labels = np.zeros((self.length,))
        
        for i in range(self.length):
            self.labels[i] = np.argmin(np.sum(np.square(np.ones((self.Nc,1)) * dataset[i,:] - self.centers),1))
        
        for i in range(self.Nc):
            self.num_per_class[i] = self.labels[self.labels == i].size
  
            
    def Merge_Small_Class(self,dataset):
        temp = self.Nc
        for i in range(self.Nc):
            indice = temp - i - 1
            if self.num_per_class[indice] < self.Theta_n:
                self.centers = np.delete(self.centers,indice,axis=0)
                self.Nc -= 1
                print('Merge Small Class!!!')
        self.Labelling(dataset)
        
        
    def Clustering(self,dataset):
        centers = np.zeros_like(self.centers)
        for i in range(self.Nc):
            cluster_class = dataset[self.labels == i]
            self.centers[i] = np.mean(cluster_class,0)
            
            [length,width] = cluster_class.shape
            Temp =  np.sum(np.sqrt(np.sum(np.square(cluster_class - self.centers[i]),1)))
            self.MeanDistanceInClass[i] = Temp / length
            self.MeanDistanceTotal += Temp
            
        
        self.MeanDistanceTotal = self.MeanDistanceTotal / self.length
        

    def Operation_Division(self):
        print('Enter Operation Division Function')
        self.StandardError = np.zeros((self.Nc,self.width))
        for i in range(self.Nc):
            cluster_class = dataset[self.labels == i]
            self.StandardError[i,:] = np.sqrt(np.mean(np.square(cluster_class - self.centers[i]),0))
        
        MaxStandardErrorVector = np.max(self.StandardError,1)
        
        flag = 1
        for i in range(self.Nc):
            if (MaxStandardErrorVector[i] > self.Theta_s):
                if (self.Nc <= self.n_class/2) or (self.MeanDistanceInClass[i] > self.MeanDistanceTotal and self.num_per_class[i] > 2*(self.Theta_n + 1)):
                    flag = 0
                    indice = np.argmax(self.StandardError[i,:])
                    new_center1 = self.centers[i,:]
                    new_center2 = self.centers[i,:]
                    new_center1[indice] += self.r *  MaxStandardErrorVector[i]
                    new_center2[indice] -= self.r *  MaxStandardErrorVector[i]
                    self.centers = np.delete(self.centers,i,axis=0)
                    self.centers = np.append(self.centers,[new_center1],axis=0)
                    self.centers = np.append(self.centers,[new_center2],axis=0)
                    self.Nc += 1
                    print('Nc = ',self.Nc)
        return flag
        
    def Operation_Merging(self):
        print('Enter Operation Merging Function')
        Distance_Center2Center = np.ones((self.Nc,self.Nc)) * 1000000
        for i in range(self.Nc):
            for j in range(self.Nc):
                if j > i :
                    distance = np.sqrt(np.sum(np.square(self.centers[i]- self.centers[j])))
                    Distance_Center2Center[i][j] = distance 
                    
        temp = Distance_Center2Center[Distance_Center2Center < self.Theta_c]
        temp = np.sort(temp)
        if temp.size > 0:
            indice = np.argwhere(Distance_Center2Center == temp[0])
            indice1 = indice[0][0]
            indice2 = indice[0][1]
            
            num =  self.num_per_class[indice1] + self.num_per_class[indice2]
            total = self.centers[indice1,:] * self.num_per_class[indice1] + self.centers[indice2,:] * self.num_per_class[indice2]
            new_center = total / num
            self.centers = np.delete(self.centers,indice2,axis=0)
            self.centers = np.delete(self.centers,indice1,axis=0)
            self.centers = np.append(self.centers,[new_center],axis=0)
                                                                                                                    
            self.Nc -= 1
            print('Nc = ',self.Nc)
    
    def Interaction(self):
        print('current algo parameters:\n')
        print('self.Nc: ',self.Nc)
        print('self.centers：\n',self.centers)
        print('n_class: ',self.n_class)
        print('Theta_n: ',self.Theta_n)
        print('Theta_s: ',self.Theta_s)
        print('Theta_c：',self.Theta_c)
        print('L: ',self.L)
        
        print('please input new parameters:\n')
        self.n_class = float(input('new k:'))
        self.Theta_n = float(input('new Theta_n:'))
        self.Theta_s = float(input('new Theta_s:'))
        self.Theta_c = float(input('new Theta_c:'))
        self.L = float(input('new L:'))
        
        
    def Training(self,dataset,iteration):
        for i in range(iteration):
            print('iteration:',i)
            FlagChange = 0
            if i> 1 and  i%5 == 0:
                FlagChange = int(input('Change Parameters?[1\Yes|0|No]'))    
                if FlagChange == 1:
                    self.Interaction()
            self.Labelling(dataset)
            self.Merge_Small_Class(dataset)
            self.Clustering(dataset)
            if i == iteration -1 : 
                self.Theta_c = 0
                self.Operation_Merging()
                break
            
            if self.Nc <= self.n_class/2 or (self.Nc < 2*self.n_class and i%2 == 1):
                flag = self.Operation_Division()
                print('flag:',flag)
                if flag == 0 :
                    continue
            if self.Nc >= 2*self.n_class or (i%2 == 0):
                self.Operation_Merging()
            
        

In [14]:
#定义取初始聚类中心的重要变量
Nc = 6
r = 0.4
Dt = 3
k = 3
Theta_n = 10
Theta_s = 0.6
Theta_c = 3
L = 2
#导入数据集计算初始聚类中心
#dataset = iris.data[0:100,:]
#dataset = np.vstack((iris.data[0:50,:],iris.data[100:150,:]))
dataset = iris.data[50:150,:]
#dataset = iris.data
[length,width] = dataset.shape
#给出了五种不同的取初始聚类中心的方法
center = Initial_Center(Nc)
#center.Random_Center(iris.data)
#center.FarDistance_Center(dataset)
center.Dense_Center(dataset,r,Dt)
print('initial_centers indices: ',center.indices)
print('initial_centers: \n',center.initial_center)
#Centers = np.zeros((k,width))
#center.Empirical_Center(Centers)

initial_centers indices:  [29, 55, 42, 56, 45, 53]
initial_centers: 
 [[5.6 2.7 4.2 1.3]
 [6.8 3.2 5.9 2.3]
 [6.9 3.2 5.7 2.3]
 [6.7 3.3 5.7 2.5]
 [6.7 3.3 5.7 2.1]
 [6.7 3.1 5.6 2.4]]


In [15]:
#创建isodata_test1 类并初始化
centers = center.initial_center
Iteration = 4
isodata_test1 = ISODATA(centers,Nc,k,Theta_n, Theta_s,Theta_c, L)
isodata_test1.Training(dataset,Iteration)

iteration: 0
Merge Small Class!!!
Merge Small Class!!!
Enter Operation Merging Function
Nc =  3
iteration: 1
Enter Operation Division Function
flag: 1
iteration: 2
Enter Operation Merging Function
Nc =  2
iteration: 3
Enter Operation Merging Function


In [16]:
print(isodata_test1.centers)

[[7.12272727 3.11363636 6.03181818 2.13181818]
 [6.01923077 2.80384615 4.58846154 1.5474359 ]]


In [17]:
print(isodata_test1.labels.reshape((2,50)))

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1.]
 [0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1.
  0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1.
  1. 1.]]
